In [28]:
from flask import Flask, render_template, redirect, jsonify
from flask_pymongo import PyMongo
from sodapy import Socrata
import time
import json
import requests
from bson import json_util


In [29]:
# establish a connection to mongodb and create table ny_potholes
mongo = PyMongo(app, uri="mongodb://localhost:27017/ny_potholes")

In [31]:
def getData():

    client = Socrata("data.cityofnewyork.us", None)
    results = client.get("fed5-ydvq", limit=2000)
    #insert data into mongodb 
    new_result = results
    print(new_result)

    # rediret to data.html and update table
    time.sleep(25)
    mongo.db.data.insert_many(new_result)
    mongo.db.data.delete_many({"location": {"$exists": False}})

getData()


[{'city': 'Elmhurst', 'latitude': '40.74253285300033', 'intersection_street_2': 'MACNISH STREET', 'descriptor': 'Pothole', 'borough': 'QUEENS', 'intersection_street_1': 'ELMHURST AVENUE', 'complaint_type': 'Street Condition', 'x_coordinate_state_plane_': '1017409', 'facility_type': 'N/A', 'incident_zip': '11373', 'longitude': '-73.88033727759354', 'y_coordinate_state_plane_': '209824', 'park_facility_name': 'Unspecified', 'agency_name': 'Department of Transportation', 'agency': 'DOT', 'address_type': 'INTERSECTION', 'unique_key': '42752368', 'resolution_action_updated_date': '2019-05-23T23:40:43', 'park_borough': 'QUEENS', 'community_board': '04 QUEENS', 'location': {'latitude': '40.74253285300033', 'human_address': '{"address":"","city":"","state":"","zip":""}', 'needs_recoding': False, 'longitude': '-73.88033727759354'}, 'created_date': '2019-05-23T23:40:43', 'status': 'Open'}, {'city': 'BROOKLYN', 'latitude': '40.60934277600126', 'descriptor': 'Pothole', 'borough': 'BROOKLYN', 'stre

In [73]:
import pandas as pd
db_data  = list(mongo.db.data.find())
cursor = mongo.db.data.find()
# Expand the cursor and construct the DataFrame
df =  pd.DataFrame(list(cursor))


# test = pd.DataFrame(list(test.find()))

# db_data  = list(mongo.db.data.find())

In [112]:
df['location']

0       {'latitude': '40.68046187636432', 'human_addre...
1       {'latitude': '40.687917777834635', 'human_addr...
2       {'latitude': '40.68224815541827', 'human_addre...
3       {'latitude': '40.67964413733879', 'human_addre...
4       {'latitude': '40.66483372793382', 'human_addre...
5       {'latitude': '40.68230833427505', 'human_addre...
6       {'latitude': '40.82113170383319', 'human_addre...
7       {'latitude': '40.84112408399198', 'human_addre...
8       {'latitude': '40.73272523122935', 'human_addre...
9       {'latitude': '40.694604249941385', 'human_addr...
10      {'latitude': '40.693700307216595', 'human_addr...
11      {'latitude': '40.66685686759046', 'human_addre...
12      {'latitude': '40.778731450159455', 'human_addr...
13      {'latitude': '40.72817350059024', 'human_addre...
14      {'latitude': '40.68102226073709', 'human_addre...
15      {'latitude': '40.85331734341858', 'human_addre...
16      {'latitude': '40.84704232114541', 'human_addre...
17      {'lati

In [117]:
len(df.index)


3802

In [76]:
selcol_df = df[['created_date', 'closed_date','borough', 'city', 'status', 'resolution_action_updated_date']]
selcol_df.tail()

,created_date,closed_date,borough,city,status,resolution_action_updated_date
3797,2019-05-14T09:03:05,2019-05-20T13:35:00,BROOKLYN,BROOKLYN,Closed,2019-05-20T13:35:00
3798,2019-05-14T08:40:36,2019-05-22T09:55:00,QUEENS,Whitestone,Closed,2019-05-22T09:55:00
3799,2019-05-14T08:37:19,2019-05-20T12:55:00,QUEENS,Ozone Park,Closed,2019-05-20T12:55:00
3800,2019-05-14T08:28:16,2019-05-21T13:15:00,STATEN ISLAND,STATEN ISLAND,Closed,2019-05-21T13:15:00
3801,2019-05-14T08:19:27,2019-05-21T12:40:00,QUEENS,Flushing,Closed,2019-05-21T12:40:00


In [87]:
# selcol_df=selcol_df[selcol_df.borough.notnull()]
selcol_df = selcol_df[selcol_df.borough != "Unspecified"]



In [108]:
df2 = pd.DataFrame(selcol_df.groupby(["borough", "status"]).size()).reset_index()
df2


,borough,status,0
0,BRONX,Closed,221
1,BRONX,Open,162
2,BRONX,Pending,14
3,BROOKLYN,Closed,455
4,BROOKLYN,Open,386
5,MANHATTAN,Closed,455
6,MANHATTAN,Open,54
7,QUEENS,Closed,854
8,QUEENS,Open,661
9,QUEENS,Pending,1


In [109]:
df2.rename(index=str, columns={0: "count"})

,borough,status,count
0,BRONX,Closed,221
1,BRONX,Open,162
2,BRONX,Pending,14
3,BROOKLYN,Closed,455
4,BROOKLYN,Open,386
5,MANHATTAN,Closed,455
6,MANHATTAN,Open,54
7,QUEENS,Closed,854
8,QUEENS,Open,661
9,QUEENS,Pending,1


In [110]:
df2[df2['status']=="Closed"]

,borough,status,0
0,BRONX,Closed,221
3,BROOKLYN,Closed,455
5,MANHATTAN,Closed,455
7,QUEENS,Closed,854
10,STATEN ISLAND,Closed,167


In [106]:
df2['borough']

0             BRONX
1             BRONX
2             BRONX
3          BROOKLYN
4          BROOKLYN
5         MANHATTAN
6         MANHATTAN
7            QUEENS
8            QUEENS
9            QUEENS
10    STATEN ISLAND
11    STATEN ISLAND
12    STATEN ISLAND
Name: borough, dtype: object